<a href="https://colab.research.google.com/github/dung18520632/Deep_Learning-Python-Basics/blob/main/Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import librosa as lb
import soundfile as sf
import numpy as np
import os, glob, pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
!unzip -uq "/content/drive/MyDrive/Colab Notebooks/archive.zip" 

In [56]:
emotion_labels = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

focused_emotion_labels = ['happy', 'sad', 'angry']

In [65]:
from librosa.core import istft
def audio_features(file_title, mfcc, chroma, mel):
    with sf.SoundFile(file_title) as audio_recording:
        audio = audio_recording.read(dtype="float32")
        sample_rate = audio_recording.samplerate
        
        if chroma:
            stft=np.abs(lb.stft(audio))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(lb.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(lb.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(lb.feature.melspectrogram(y=audio, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        return result

In [74]:
def loading_audio_data():
    x = []
    y = []
    for file in glob.glob("/content//Actor_*//*.wav"):
        file_path=os.path.basename(file)
        emotion = emotion_labels[file_path.split("-")[2]]
        if emotion not in focused_emotion_labels:
            continue
        feature = audio_features(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    final_dataset = train_test_split(np.array(x), y, test_size=0.25, random_state=42)
    return final_dataset

In [75]:
X_train, X_test, y_train, y_test = loading_audio_data()

/content/Actor_09/03-01-07-01-01-01-09.wav
/content/Actor_09/03-01-04-01-01-02-09.wav
/content/Actor_09/03-01-06-02-01-02-09.wav
/content/Actor_09/03-01-06-01-02-01-09.wav
/content/Actor_09/03-01-05-02-02-01-09.wav
/content/Actor_09/03-01-01-01-02-02-09.wav
/content/Actor_09/03-01-05-02-02-02-09.wav
/content/Actor_09/03-01-05-01-02-01-09.wav
/content/Actor_09/03-01-08-01-01-01-09.wav
/content/Actor_09/03-01-07-02-01-02-09.wav
/content/Actor_09/03-01-02-02-02-01-09.wav
/content/Actor_09/03-01-02-02-01-02-09.wav
/content/Actor_09/03-01-07-02-02-01-09.wav
/content/Actor_09/03-01-07-01-02-01-09.wav
/content/Actor_09/03-01-03-01-02-02-09.wav
/content/Actor_09/03-01-04-02-01-01-09.wav
/content/Actor_09/03-01-03-01-01-01-09.wav
/content/Actor_09/03-01-03-01-02-01-09.wav
/content/Actor_09/03-01-07-01-01-02-09.wav
/content/Actor_09/03-01-03-02-01-02-09.wav
/content/Actor_09/03-01-06-02-02-01-09.wav
/content/Actor_09/03-01-03-02-02-02-09.wav
/content/Actor_09/03-01-08-01-02-02-09.wav
/content/Ac

In [76]:
model = MLPClassifier(hidden_layer_sizes=(200,), learning_rate='adaptive', max_iter=400)

In [77]:
model.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [79]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy of the Recognizer is: {:.1f}%".format(accuracy*100))

Accuracy of the Recognizer is: 67.4%
